In [62]:
# Imports
from xml.dom import minidom
import pandas as pd
import re
from datetime import datetime

#pega a data corrente para inserir na coluna Data de Lançamento e extrai o mes para tratar e colocar na coluna Mês
today = datetime.now()
today = today.strftime('%Y-%m-%d')
mes = today[5:7]

# Tratamento para a coluna Mês
if mes == '01':
    mes = 'JANEIRO'
elif mes == '02':
    mes = 'FEVEREIRO'
elif mes == '03':
    mes = 'MARÇO'
elif mes == '04':
    mes = 'ABRIL'
elif mes == '05':
    mes = 'MAIO'
elif mes == '06':
    mes = 'JUNHO'
elif mes == '07':
    mes = 'JULHO'
elif mes == '08':
    mes = 'AGOSTO'
elif mes == '09':
    mes = 'SETEMBRO'
elif mes == '10':
    mes = 'OUTUBRO'
elif mes == '11':
    mes = 'NOVEMBRO'
else:
    mes = 'DEZEMBRO'

# Criação de path do arquivo xml e excel para tratamento.
# Incluir o caminho do arquivo. Ex: C:/documents/user - sem a barra final
# incluir o nome do arquivo sem a extensão
path = str(input('coloque o caminho do arquivo XML:'))
path = path.replace('\\', '/')
arquivo = str(input('Digite o nome do arquivo XML:'))
arquivo = path +'/'+ arquivo + '.xml'
path2 = str(input('coloque o caminho do arquivo EXCEL:'))
path2 = path2.replace('\\', '/')
arquivo2 = str(input('Digite o nome do arquivo EXCEL:'))
arquivo2 = path2 +'/'+ arquivo2 + '.xlsx'
aba_planilha = str(input('Digite o nome da Aba da planilha para inserir a informacao:'))

# leitura do arquivo Excel
exel = pd.read_excel(arquivo2, sheet_name= aba_planilha)

# Leitura do arquivo xml e extração dos dados por meio das tags
with open(arquivo, 'r') as f:
    xml = minidom.parse(f)
    dtEmi = xml.getElementsByTagName('dhEmi')
    RSocial = xml.getElementsByTagName('xNome')
    NFantasia = xml.getElementsByTagName('xFant')
    qtde = xml.getElementsByTagName('qCom')
    peca = xml.getElementsByTagName('xProd')
    vl_tot = xml.getElementsByTagName('vProd')
    nfe = xml.getElementsByTagName('nNF')
    desconto = xml.getElementsByTagName('vDesc')
    vl_final = xml.getElementsByTagName('vNF')

'''Segundo o negócio, foi necessário incluir mais de uma informação em uma mesma coluna, 
pois, estavam inputando informações por numeros de NFE e não por SKU. Assim foi realizado este tratamento personalizado para a necessidade do cliente.'''
peca2 = []
count = 0
if len(peca) > 1:
    for i in peca:
        dado = i.firstChild.data
        peca2.append(dado)
    separador = ' + '
    peca = separador.join(peca2)
    qtde2 = []
    count = 0
    for i in qtde:
        dado = i.firstChild.data
        qtde2.append(float(dado))
    qtde = sum(qtde2)
    vl_tot2 = []
    count = 0
    for i in vl_tot:
        dado = i.firstChild.data
        vl_tot2.append(float(dado))
    vl_tot = sum(vl_tot2)
    desconto2 = []
    count = 0
    for i in desconto:
        dado = i.firstChild.data
        desconto2.append(float(dado))
    desconto = sum(desconto2)
else: 
    peca = peca[0].firstChild.data
    qtde = qtde[0].firstChild.data
    vl_tot = vl_tot[0].firstChild.data
    desconto = desconto[0].firstChild.data


#joga dados em uma lista para extrair as informações
lista = [dtEmi, RSocial, NFantasia, nfe, vl_final]

#checa se existe algum dado vazio, se houver irá colocar o item como sem informação
lista_elementos = []
for i in lista:
    if len(i) > 0:
        dado = i[0].firstChild.data
        lista_elementos.append(dado)
    else:
        dado = 'sem informação'
        lista_elementos.append(dado)

lista_elementos.append(peca)
lista_elementos.append(qtde)
lista_elementos.append(vl_tot)
lista_elementos.append(desconto)
lista_elementos[0] = lista_elementos[0][:10]

# insere uma linha no arquivo e grava as indormações conforme o nome da coluna
elementos = {'MÊS': mes,
                 'DATA DE LAÇAMENTO': today,
                 'DATA EMISSÃO':lista_elementos[0], 
                 'RAZÃO SOCIAL': lista_elementos[1], 
                 'NOME FANTASIA': lista_elementos[2], 
                 'QTDE': lista_elementos[6], 
                 'PEÇAS': lista_elementos[5], 
                 'VALOR  TOTAL': lista_elementos[7], 
                 'NFE / RECIBO': lista_elementos[3],
                 'DESCONTO': lista_elementos[8], 
                 'VALOR FINAL': lista_elementos[4]}

exel = exel.append(elementos, ignore_index=True)
# Tratamento de Erro e saving das informações no arquivo
try:
    with pd.ExcelWriter(arquivo2, mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
        exel.to_excel(writer, sheet_name = aba_planilha, index=False)
        print('Arquivos inseridos com sucesso!')
except:
    print('Arquivos não inseridos, contate o suporte!')


C:\Users\Nivea Helena\AppData\Local\Temp\ipykernel_3616\2907171938.py:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  exel = exel.append(elementos, ignore_index=True)


In [78]:
try:
    with pd.ExcelWriter(arquivo2, mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
        exel.to_excel(writer, sheet_name = aba_planilha, index=False)
        print('Arquivos inseridos com sucesso!')
except:
    print('Arquivos não inseridos, contate o suporte!')

Arquivos inseridos com sucesso!


In [77]:
exel = exel.drop(index=0)

In [74]:
exel = pd.read_excel('C:/Users/Nivea Helena/Documents/developer/teste/teste_excel.xlsx', sheet_name= aba_planilha)
exel

,MÊS,DATA DE LAÇAMENTO,DATA EMISSÃO,CONTA,RAZÃO SOCIAL,NOME FANTASIA,PLACA,MARCA,MODELO,QTDE,...,NFE / RECIBO,DESCONTO,VALOR FINAL,TOTAL %,SERVIÇO,Área de Manutenção,Tipo de Manutenção,Tipo de Despsa,VENCIMENTO,DATA DE ENTREGA AO FINANCEIRO
0,SETEMBRO,2023-09-30,2023-07-27,NaN,JOHN ENDERSON BERNARDO SOARES,OFICINA JM,NaN,NaN,NaN,1.0,...,447,0.0,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
'C:/Users/Nivea Helena/Documents/developer/teste/teste_excel.xlsx'

SyntaxError: unexpected character after line continuation character (2049999177.py, line 1)